In [87]:
from tools.transcripts import *
import pysam
from tools.procOps import *
from collections import *
from tools.bio import *

In [26]:
recs = get_transcript_dict('notch2nl_hg38.bed')

In [123]:
def get_region(r, bam):
    region = '{}:{}-{}'.format(r.chromosome, r.start + 1, r.stop)
    cmd = [['samtools', 'mpileup', '-r', region, '-uv', '-f', 
           '/hive/groups/recon/10x_genomics/references/refdata-hsapiens-hg38/fasta/genome.fa', bam],
           ['grep', '-v', '"#"'],
           ['awk', '{printf $4}']]
    return call_proc_lines(cmd)[0].split('REF')[1]

full_results = {}
for p in xrange(4):
    bam = 'c{0}/c{0}.hg38.bam'.format(p)
    results = defaultdict(dict)
    for para, para_tx in recs.iteritems():
        if para_tx.strand == '-':
            exons = para_tx.exon_intervals[::-1]
        else:
            exons = para_tx.exon_intervals
        for i, exon in enumerate(exons, 1):
            s = get_region(exon, bam)
            if para_tx.strand == '-':
                s = reverse_complement(s)
            results[para][i] = s
    # reshape to produce a list by-exon
    by_exon = defaultdict(list)
    for para in results:
        for i, s in results[para].iteritems():
            if len(s) > 0:
                by_exon[i].append(s)
    assert all(len(x) == 1 for x in by_exon.itervalues())
    full_results[p] = ''.join(by_exon[i][0] for i in range(1, 6, 1))

In [124]:
with open('transcripts.fa', 'w') as outf:
    for c, s in full_results.iteritems():
        write_fasta(outf, 'c{}-assembly'.format(c), s)